# Multi Factor Optimization
1. Read factor ranking from files.
2. Assign weight for each factor
3. Rank again with assigned weights for each factors
4. Get Return based on rank


# Read factor ranking from files

In [460]:
import pandas as pd
import numpy as np
import sqlalchemy
from urllib.parse import urlencode
import requests
from requests import Response
import datetime as dt
from datetime import time, date, timedelta
from dateutil.relativedelta import relativedelta
import itertools
from math import ceil
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

In [546]:
rank1 = pd.read_csv('ranks/Valuation/total equity_rank.csv') 
rank2 = pd.read_csv('ranks/Growth/operating_profit_growth_rank.csv')
monthly_ret = pd.read_csv('ranks/monthly_ret.csv')
opt_result=pd.DataFrame(columns=['weight1','weight2','cumul_return','mdd','return_mdd'])
print(opt_result)

Empty DataFrame
Columns: [weight1, weight2, cumul_return, mdd, return_mdd]
Index: []


In [547]:
def rank_opt(rank1, rank2, init, unit,opt_result):
    rank1 = rank1
    rank2 = rank2
    iterations = int(1/unit)
    
    counter=0
    
    for x in range(1, iterations) :
        weight1 = init + unit*counter
        weight2 = 1-weight1
        counter=counter + 1 
        rank_sum = rank1*weight1 + rank2*weight2
        rank_sum.yr_month=rank1.yr_month
        rank_rank=rank_sum.rank(axis=1, ascending=True, pct=True)
        rank_rank.yr_month=rank1.yr_month
        signal=rank_rank[rank_rank<=0.1].dropna(axis=1, how='all')
        signal[signal>0]=1
        signal.yr_month=rank1.yr_month
        #signal.to_csv('ranks/multi_factor_signal.csv')
        signal=signal.shift(1)
        mm_hold=signal.count(axis=1).max()
        profit_arr=(signal*monthly_ret[signal.columns.tolist()]).dropna(axis=1, how='all')


        profit_arr['daily_ret']=profit_arr.mean(axis=1)

        profit_arr['profit']=(profit_arr['daily_ret']+1).cumprod()
        daily_profit=profit_arr['daily_ret'].tolist()
        cumulative=profit_arr['profit'].dropna().tolist()
        #print(daily_profit)
        profit_arr['yr_month']=rank1.yr_month

        win_arr = [x for x in daily_profit if x > 0]
        loss_arr=[x for x in daily_profit if x < 0]
        win_rate = len(win_arr) / (len(loss_arr)+len(win_arr))
        dd_arr=[]
        for i in range(len(cumulative)):
            if i>0:
                dd_arr.append((max(cumulative[:i])- cumulative[i-1])/max(cumulative[:i]))
        max_dd=max(dd_arr)
        print('weight1:{} and weight2:{}'.format(weight1,weight2))
        print('CUMULATIVE PROFIT: {}'.format(round(cumulative[-1],4)))
        print('MDD: {}' .format(max_dd))
        print("mm_holding:{}------winrate:{}-----profit/MDD:{}".format(mm_hold, round(win_rate,4), round(cumulative[-1]/max_dd,4)))
        new_low ={'weight1':weight1, 'weight2':weight2,'cumul_return':round(cumulative[-1],4),'mdd':max_dd,'return_mdd':round(cumulative[-1]/max_dd,4)}
        opt_result = opt_result
        opt_result= opt_result.append(new_low,ignore_index=True)
        opt_result.to_csv("Ranks/opt_result.csv")
    return round(cumulative[-1],4)

In [548]:
rank_opt(rank1, rank2, 0.01,0.01,opt_result)

weight1:0.01 and weight2:0.99
CUMULATIVE PROFIT: 7.2159
MDD: 0.1814443970053572
mm_holding:34------winrate:0.7051-----profit/MDD:39.7695
weight1:0.02 and weight2:0.98
CUMULATIVE PROFIT: 6.9798
MDD: 0.1781861868172605
mm_holding:34------winrate:0.6923-----profit/MDD:39.1715
weight1:0.03 and weight2:0.97
CUMULATIVE PROFIT: 6.9077
MDD: 0.18282749417480604
mm_holding:34------winrate:0.6795-----profit/MDD:37.7827
weight1:0.04 and weight2:0.96
CUMULATIVE PROFIT: 6.9239
MDD: 0.18135060616446777
mm_holding:34------winrate:0.6795-----profit/MDD:38.1799
weight1:0.05 and weight2:0.95
CUMULATIVE PROFIT: 7.3384
MDD: 0.18135060616446783
mm_holding:34------winrate:0.6795-----profit/MDD:40.4651
weight1:0.060000000000000005 and weight2:0.94
CUMULATIVE PROFIT: 6.9646
MDD: 0.20051163243525214
mm_holding:34------winrate:0.6923-----profit/MDD:34.7339
weight1:0.06999999999999999 and weight2:0.93
CUMULATIVE PROFIT: 7.0253
MDD: 0.20051163243525214
mm_holding:34------winrate:0.6923-----profit/MDD:35.037
weight

weight1:0.6900000000000001 and weight2:0.30999999999999994
CUMULATIVE PROFIT: 15.5093
MDD: 0.17658006953180266
mm_holding:34------winrate:0.7308-----profit/MDD:87.8315
weight1:0.7000000000000001 and weight2:0.29999999999999993
CUMULATIVE PROFIT: 15.0494
MDD: 0.174559841451938
mm_holding:34------winrate:0.7308-----profit/MDD:86.2135
weight1:0.7100000000000001 and weight2:0.2899999999999999
CUMULATIVE PROFIT: 14.2595
MDD: 0.17455984145193806
mm_holding:34------winrate:0.7051-----profit/MDD:81.6886
weight1:0.72 and weight2:0.28
CUMULATIVE PROFIT: 14.4068
MDD: 0.17513654272068077
mm_holding:34------winrate:0.7051-----profit/MDD:82.2606
weight1:0.73 and weight2:0.27
CUMULATIVE PROFIT: 14.9738
MDD: 0.17513654272068077
mm_holding:34------winrate:0.7051-----profit/MDD:85.498
weight1:0.74 and weight2:0.26
CUMULATIVE PROFIT: 14.9074
MDD: 0.17513654272068083
mm_holding:34------winrate:0.6923-----profit/MDD:85.1188
weight1:0.75 and weight2:0.25
CUMULATIVE PROFIT: 14.437
MDD: 0.17513654272068074
mm

8.6574

In [549]:
opt_result = pd.read_csv('ranks/opt_result.csv') 

In [550]:
print(opt_result)

    Unnamed: 0  weight1  weight2  cumul_return       mdd  return_mdd
0            0     0.01     0.99        7.2159  0.181444     39.7695
1            1     0.02     0.98        6.9798  0.178186     39.1715
2            2     0.03     0.97        6.9077  0.182827     37.7827
3            3     0.04     0.96        6.9239  0.181351     38.1799
4            4     0.05     0.95        7.3384  0.181351     40.4651
..         ...      ...      ...           ...       ...         ...
94          94     0.95     0.05       10.0527  0.166052     60.5395
95          95     0.96     0.04       10.1642  0.175405     57.9470
96          96     0.97     0.03        9.1283  0.190988     47.7951
97          97     0.98     0.02        9.4083  0.185365     50.7552
98          98     0.99     0.01        8.6574  0.234596     36.9035

[99 rows x 6 columns]


In [551]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=opt_result['weight1'].tolist(), y=opt_result['cumul_return'].tolist(),
                                 mode='lines',
                                 name='cumulative'))

fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=opt_result['weight1'].tolist(), y=opt_result['return_mdd'].tolist(),
                                 mode='lines',
                                 name='cumulative'))

fig.show()

In [554]:
weight1 = 0.82
weight2 = 0.18

rank_sum = rank1*weight1 + rank2*weight2
rank_sum.yr_month=rank1.yr_month
rank_rank=rank_sum.rank(axis=1, ascending=True, pct=True)
rank_rank.yr_month=rank1.yr_month
signal=rank_rank[rank_rank<=0.1].dropna(axis=1, how='all')
signal[signal>0]=1
signal.yr_month=rank1.yr_month
#signal.to_csv('ranks/multi_factor_signal.csv')
signal=signal.shift(1)
mm_hold=signal.count(axis=1).max()
profit_arr=(signal*monthly_ret[signal.columns.tolist()]).dropna(axis=1, how='all')

signal.to_csv('equity_netprofitgrowth_signal.csv')
profit_arr['daily_ret']=profit_arr.mean(axis=1)

profit_arr['profit']=(profit_arr['daily_ret']+1).cumprod()
daily_profit=profit_arr['daily_ret'].tolist()
cumulative=profit_arr['profit'].dropna().tolist()
#print(daily_profit)
profit_arr['yr_month']=rank1.yr_month

win_arr = [x for x in daily_profit if x > 0]
loss_arr=[x for x in daily_profit if x < 0]
win_rate = len(win_arr) / (len(loss_arr)+len(win_arr))
dd_arr=[]
for i in range(len(cumulative)):
    if i>0:
        dd_arr.append((max(cumulative[:i])- cumulative[i-1])/max(cumulative[:i]))
max_dd=max(dd_arr)
print('weight1:{} and weight2:{}'.format(weight1,weight2))
print('CUMULATIVE PROFIT: {}'.format(round(cumulative[-1],4)))
print('MDD: {}' .format(max_dd))
print("mm_holding:{}------winrate:{}-----profit/MDD:{}".format(mm_hold, round(win_rate,4), round(cumulative[-1]/max_dd,4)))
fig = go.Figure()
fig.add_trace(go.Scatter(x=profit_arr['yr_month'].astype(str).tolist(), y=profit_arr['profit'].tolist(),
                             mode='lines',
                             name='cumulative'))

fig.show()



weight1:0.82 and weight2:0.18
CUMULATIVE PROFIT: 14.3631
MDD: 0.16793325178060123
mm_holding:34------winrate:0.6795-----profit/MDD:85.5289


Calculation of profit based on generated signal
